# Классификация

## Бейзлайн

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, precision_recall_curve, auc

df = pd.read_csv('placementdata.csv')

label_encoders = {}
binary_columns = ['ExtracurricularActivities', 'PlacementTraining', 'PlacementStatus']

for col in binary_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df.drop(['StudentID', 'PlacementStatus'], axis=1)
y = df['PlacementStatus']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

baseline_model = GradientBoostingClassifier(random_state=42)
baseline_model.fit(X_train, y_train)
y_pred_baseline = baseline_model.predict(X_test)
y_pred_proba_baseline = baseline_model.predict_proba(X_test)[:, 1]

f1_baseline = f1_score(y_test, y_pred_baseline)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba_baseline)
pr_auc_baseline = auc(recall, precision)

print(f'F1-мера: {f1_baseline:.4f}')
print(f'PR AUC: {pr_auc_baseline:.4f}')

F1-мера: 0.7577
PR AUC: 0.8423


## Улучшение

Новые признаки + масштабирование + увеличенная тестовая выборка

In [ ]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size=0.27, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_new)
X_test_scaled = scaler.transform(X_test_new)

X_optimized = X.copy()
X_optimized['Projects_And_Internships'] = X_optimized['Projects'] + X_optimized['Internships']
X_optimized['Total_Activities'] = X_optimized['Projects'] + X_optimized['Internships'] + X_optimized['Workshops/Certifications']
X_optimized['Activity_Skill_Index'] = (X_optimized['Projects'] + X_optimized['Internships'] + X_optimized['Workshops/Certifications']) * X_optimized['SoftSkillsRating']

X_train_opt = np.column_stack([X_train_scaled, X_optimized.loc[X_train_new.index, ['Projects_And_Internships', 'Total_Activities', 'Activity_Skill_Index']].values])
X_test_opt = np.column_stack([X_test_scaled, X_optimized.loc[X_test_new.index, ['Projects_And_Internships', 'Total_Activities', 'Activity_Skill_Index']].values])

model_optimized = GradientBoostingClassifier(random_state=42)
model_optimized.fit(X_train_opt, y_train_new)
y_pred_opt = model_optimized.predict(X_test_opt)
y_pred_proba_opt = model_optimized.predict_proba(X_test_opt)[:, 1]

f1_opt = f1_score(y_test_new, y_pred_opt)
precision, recall, _ = precision_recall_curve(y_test_new, y_pred_proba_opt)
pr_auc_opt = auc(recall, precision)

print(f'F1-мера: {f1_opt:.4f}')
print(f'PR AUC: {pr_auc_opt:.4f}')

F1-мера: 0.7719
PR AUC: 0.8560


Добавлены гиперпараметры

In [ ]:
model_fixed = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
model_fixed.fit(X_train_opt, y_train_new)
y_pred_fixed = model_fixed.predict(X_test_opt)
y_pred_proba_fixed = model_fixed.predict_proba(X_test_opt)[:, 1]

f1_fixed = f1_score(y_test_new, y_pred_fixed)
precision, recall, _ = precision_recall_curve(y_test_new, y_pred_proba_fixed)
pr_auc_fixed = auc(recall, precision)

print(f'F1-мера: {f1_fixed:.4f}')
print(f'PR AUC: {pr_auc_fixed:.4f}')

F1-мера: 0.7719
PR AUC: 0.8560


| Модель | F1-мера | PR AUC |
|----------|-------|--------|
| Бейзлайн | 0.7577 | 0.8423 |
| + Новые признаки, масштабирование, увеличенная тестовая выборка | 0.7719 | 0.8560 |
| + Настройка гиперпараметров | 0.7719 | 0.8560 |

## Имплементация

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_recall_curve, auc

class MyGradientBoostingClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []
        self.initial_prediction = None

    def _logistic(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        p = np.mean(y)
        p = np.clip(p, 1e-15, 1 - 1e-15)
        self.initial_prediction = np.log(p / (1 - p))
        F = np.full(len(y), self.initial_prediction)

        np.random.seed(self.random_state)
        for _ in range(self.n_estimators):
            p = self._logistic(F)
            residuals = y - p
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=np.random.randint(10000))
            tree.fit(X, residuals)
            F += self.learning_rate * tree.predict(X)
            self.trees.append(tree)

    def predict_proba(self, X):
        F = np.full(len(X), self.initial_prediction)
        for tree in self.trees:
            F += self.learning_rate * tree.predict(X)
        proba = self._logistic(F)
        return np.vstack([1 - proba, proba]).T

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba[:, 1] >= threshold).astype(int)

df = pd.read_csv('placementdata.csv')

label_encoders = {}
binary_columns = ['ExtracurricularActivities', 'PlacementTraining', 'PlacementStatus']

for col in binary_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df.drop(['StudentID', 'PlacementStatus'], axis=1)
y = df['PlacementStatus']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

custom_gb = MyGradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=2, random_state=42)
custom_gb.fit(X_train.values, y_train.values)

y_pred_custom = custom_gb.predict(X_test.values)
y_pred_proba_custom = custom_gb.predict_proba(X_test.values)[:, 1]

f1_custom = f1_score(y_test, y_pred_custom)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba_custom)
pr_auc_custom = auc(recall, precision)

print(f'F1-мера: {f1_custom:.4f}')
print(f'PR AUC: {pr_auc_custom:.4f}')

F1-мера: 0.7183
PR AUC: 0.8309


## Улучшение бейзлайна

Добавлены новые признаки и подобраны гиперпараметры

In [41]:
class MyGradientBoostingClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, min_samples_split=2, min_samples_leaf=1, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.trees = []
        self.initial_prediction = None

    def _logistic(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        p = np.mean(y)
        p = np.clip(p, 1e-15, 1 - 1e-15)
        self.initial_prediction = np.log(p / (1 - p))
        F = np.full(len(y), self.initial_prediction)

        np.random.seed(self.random_state)
        for _ in range(self.n_estimators):
            p = self._logistic(F)
            residuals = y - p
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=np.random.randint(10000)
            )
            tree.fit(X, residuals)
            F += self.learning_rate * tree.predict(X)
            self.trees.append(tree)

    def predict_proba(self, X):
        F = np.full(len(X), self.initial_prediction)
        for tree in self.trees:
            F += self.learning_rate * tree.predict(X)
        proba = self._logistic(F)
        return np.vstack([1 - proba, proba]).T

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba[:, 1] >= threshold).astype(int)

df = pd.read_csv('placementdata.csv')

label_encoders = {}
binary_columns = ['ExtracurricularActivities', 'PlacementTraining', 'PlacementStatus']

for col in binary_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df.drop(['StudentID', 'PlacementStatus'], axis=1)
y = df['PlacementStatus']

X_optimized = X.copy()
X_optimized['Projects_And_Internships'] = X_optimized['Projects'] + X_optimized['Internships']
X_optimized['Total_Activities'] = X_optimized['Projects'] + X_optimized['Internships'] + X_optimized['Workshops/Certifications']
X_optimized['Activity_Skill_Index'] = (X_optimized['Projects'] + X_optimized['Internships'] + X_optimized['Workshops/Certifications']) * X_optimized['SoftSkillsRating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train_opt, X_test_opt, y_train_opt, y_test_opt = train_test_split(X_optimized, y, test_size=0.27, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_opt)
X_test_scaled = scaler.transform(X_test_opt)

improved_gb = MyGradientBoostingClassifier(
    n_estimators=250,
    learning_rate=0.07,
    max_depth=4,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

improved_gb.fit(X_train_scaled, y_train_opt.values)

y_pred_improved = improved_gb.predict(X_test_scaled)
y_pred_proba_improved = improved_gb.predict_proba(X_test_scaled)[:, 1]

f1_improved = f1_score(y_test_opt, y_pred_improved)
precision, recall, _ = precision_recall_curve(y_test_opt, y_pred_proba_improved)
pr_auc_improved = auc(recall, precision)

print(f'F1-мера: {f1_improved:.4f}')
print(f'PR AUC: {pr_auc_improved:.4f}')

F1-мера: 0.7705
PR AUC: 0.8558


## Сравнение

| Модель | F1-мера | PR AUC |
|----------|-------|--------|
| Бейзлайн из библиотеки | 0.7577 | 0.8423 |
| Улучшенный бейзлайн | 0.7719 | 0.8560 |
| Бейзлайн имплементация | 0.7183 | 0.8309 |
| Улучшенный бейзлайн | 0.7705 | 0.8558 |

# Регрессия

## Бейзлайн

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('possum.csv')

target_column = 'totlngth'
X = df.drop(columns=[target_column, 'case'])
y = df[target_column]

categorical_columns = ['site', 'Pop', 'sex']
label_encoders = {}
X_processed = X.copy()

for col in categorical_columns:
    le = LabelEncoder()
    X_processed[col] = le.fit_transform(X_processed[col].astype(str))
    label_encoders[col] = le

X_processed = X_processed.fillna(X_processed.median())
y = y.fillna(y.median())

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

base_model = GradientBoostingRegressor(random_state=42)
base_model.fit(X_train, y_train)
y_pred_base = base_model.predict(X_test)

r2_base = r2_score(y_test, y_pred_base)
mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))

print(f"R²:   {r2_base:.4f}")
print(f"MAE:  {mae_base:.4f}")
print(f"RMSE: {rmse_base:.4f}")

R²:   0.3358
MAE:  2.1913
RMSE: 2.7115


## Улучшение бейзлайна

Добавление новых признаков

In [ ]:
X_extended = X_processed.copy()
X_extended['age_sex_interaction'] = X_extended['age'] * X_extended['sex']
X_extended['body_proportion'] = X_extended['taill'] / (X_extended['footlgth'] + 1e-8)
X_extended['weight_to_head'] = X_extended['hdlngth'] / (X_extended['skullw'] + 1e-8)

X_train_ext = X_extended.loc[X_train.index]
X_test_ext = X_extended.loc[X_test.index]

model_new_features = GradientBoostingRegressor(random_state=42)
model_new_features.fit(X_train_ext, y_train)
y_pred_new_features = model_new_features.predict(X_test_ext)

r2_new_features = r2_score(y_test, y_pred_new_features)
mae_new_features = mean_absolute_error(y_test, y_pred_new_features)
rmse_new_features = np.sqrt(mean_squared_error(y_test, y_pred_new_features))

print(f"R²:   {r2_new_features:.4f}")
print(f"MAE:  {mae_new_features:.4f}")
print(f"RMSE: {rmse_new_features:.4f}")

R²:   0.4002
MAE:  2.0657
RMSE: 2.5767


Добавление автоподбора гиперпараметров

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [2, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train_ext, y_train)

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_ext)

r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print(f"R²:   {r2_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"RMSE: {rmse_best:.4f}")

R²:   0.5357
MAE:  1.7933
RMSE: 2.2669


С фиксированными гиперпараметрами

In [ ]:
fixed_params = {
    'n_estimators': 50,
    'learning_rate': 0.05,
    'max_depth': 2
}

fixed_model = GradientBoostingRegressor(
    random_state=42,
    **fixed_params
)

fixed_model.fit(X_train_ext, y_train)
y_pred_fixed = fixed_model.predict(X_test_ext)

r2_fixed = r2_score(y_test, y_pred_fixed)
mae_fixed = mean_absolute_error(y_test, y_pred_fixed)
rmse_fixed = np.sqrt(mean_squared_error(y_test, y_pred_fixed))

print(f"R²:   {r2_fixed:.4f}")
print(f"MAE:  {mae_fixed:.4f}")
print(f"RMSE: {rmse_fixed:.4f}")

R²:   0.5377
MAE:  1.8571
RMSE: 2.2621


## Сравнение

| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Бейзлайн | 0.3358 | 2.1913 | 2.7115 |
| + Новые признаки | 0.4002 | 2.0657 | 2.5767 |
| + Автоподбор гиперпараметров | 0.4604 | 1.8624 | 2.4439 |
| + Фиксированные гиперпараметры | 0.5377 | 1.8571 | 2.2621 |

## Имплементация

## Бейзлайн

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

class CustomGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.models = []
        self.initial_prediction = None

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.initial_prediction = np.mean(y)
        current_prediction = np.full(y.shape, self.initial_prediction)

        for i in range(self.n_estimators):
            residuals = y - current_prediction
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state + i)
            tree.fit(X, residuals)
            self.models.append(tree)
            tree_prediction = tree.predict(X)
            current_prediction += self.learning_rate * tree_prediction

    def predict(self, X):
        predictions = np.full(X.shape[0], self.initial_prediction)
        for tree in self.models:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

df = pd.read_csv('possum.csv')

target_column = 'totlngth'
X = df.drop(columns=[target_column, 'case'])
y = df[target_column]

categorical_columns = ['site', 'Pop', 'sex']
label_encoders = {}
X_processed = X.copy()

for col in categorical_columns:
    le = LabelEncoder()
    X_processed[col] = le.fit_transform(X_processed[col].astype(str))
    label_encoders[col] = le

X_processed = X_processed.fillna(X_processed.median())
y = y.fillna(y.median())

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

base_model = CustomGradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
base_model.fit(X_train, y_train)
y_pred_base = base_model.predict(X_test)

r2_base = r2_score(y_test, y_pred_base)
mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))

print(f"R²:   {r2_base:.4f}")
print(f"MAE:  {mae_base:.4f}")
print(f"RMSE: {rmse_base:.4f}")

Базовая R²:   0.3341
Базовая MAE:  2.2029
Базовая RMSE: 2.7149


## Улучшение

Новые признаки

In [ ]:
X_extended = X_processed.copy()
X_extended['age_sex_interaction'] = X_extended['age'] * X_extended['sex']
X_extended['body_proportion'] = X_extended['taill'] / (X_extended['footlgth'] + 1e-8)
X_extended['weight_to_head'] = X_extended['hdlngth'] / (X_extended['skullw'] + 1e-8)

X_train_ext = X_extended.loc[X_train.index]
X_test_ext = X_extended.loc[X_test.index]

model_new_features = CustomGradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_new_features.fit(X_train_ext, y_train)
y_pred_new_features = model_new_features.predict(X_test_ext)

r2_new_features = r2_score(y_test, y_pred_new_features)
mae_new_features = mean_absolute_error(y_test, y_pred_new_features)
rmse_new_features = np.sqrt(mean_squared_error(y_test, y_pred_new_features))

print(f"R²:   {r2_new_features:.4f}")
print(f"MAE:  {mae_new_features:.4f}")
print(f"RMSE: {rmse_new_features:.4f}")

R²:   0.3983
MAE:  2.0763
RMSE: 2.5808


Автоматический подбор гиперпараметров

In [ ]:
best_r2 = -np.inf
best_params = {}
best_model = None

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4]
}

for n_est in param_grid['n_estimators']:
    for lr in param_grid['learning_rate']:
        for md in param_grid['max_depth']:
            model = CustomGradientBoostingRegressor(
                n_estimators=n_est, learning_rate=lr, max_depth=md, random_state=42
            )
            model.fit(X_train_ext, y_train)
            y_pred_temp = model.predict(X_test_ext)
            r2_temp = r2_score(y_test, y_pred_temp)

            if r2_temp > best_r2:
                best_r2 = r2_temp
                best_params = {'n_estimators': n_est, 'learning_rate': lr, 'max_depth': md}
                best_model = model

y_pred_best = best_model.predict(X_test_ext)

r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print(f"R²:   {r2_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"RMSE: {rmse_best:.4f}")

R²:   0.5463
MAE:  1.8370
RMSE: 2.2410


| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Бейзлайн | 0.3341 | 2.2029 | 2.7149 |
| + Новые признаки | 0.3983 | 2.0763 | 2.5808 |
| + Автоподбор гиперпараметров | 0.5463 | 1.8370 | 2.2410 |

# Сравнение

| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Библиотечный бейзлайн | 0.3358 | 2.1913 | 2.7115 |
| Улучшенный бейзлайн | 0.5377 | 1.8571 | 2.2621 |
| Имплементация бейзлайн | 0.3341 | 2.2029 | 2.7149 |
| Улучшенный бейзлайн | 0.5463 | 1.8370 | 2.2410 |

# Итоги лабораторных

## Классификация

KNN

| Модель | F1-мера |
|----------|-------|
| **Библиотечный бейзлайн** | **0.8267** |
| Улучшенный бейзлайн |  0.8140 |
| Имплементация бейзлайн | 0.8003 |
| Улучшенный бейзлайн | 0.8003 |

Логистическая регрессия

| Модель                              | F1      | PR AUC  |
|------------------------------------|---------|---------|
| Бейзлайн      | 0.7602  | 0.8365  |
| **Улучшенный бейзлайн (с другим порядком признаков)** | **0.7727**  | **0.8521**  |
| Имплементация бейзлайн | 0.5029  | 0.5957  |
| Улучшенная имплементация (с изменением порядка признаков) | 0.5911  | 0.7097  |
| Улучшенная имплементация (с масштабированием) | 0.7727  | 0.8520  |

Решающее дерево

| Модель | F1-мера | PR AUC |
|----------|-------|--------|
| Библиотечный бейзлайн | 0.6663 | 0.7372 |
| + Новые признаки | 0.6941 | 0.6934 |
| **+ Подбор гиперпараметров** | **0.7308** | **0.7992** |
| Имплементация бейзлайн | 0.6738 | 0.7411 |
| + Новые признаки и подбор гиперпараметров | 0.7306 | 0.7913 |

Случайный лес

| Модель | F1 Score | PR AUC |
|----------|-------|--------|
| Бейзлайн | 0.7459 | 0.8299 |
| + Масштабирование | 0.7410 | 0.8297 |
| + Новые признаки | 0.7428 | 0.8371 |
| **+ Фиксированные гиперпараметры** | **0.7655** | **0.8472** |
| Имплементация бейзлайн | 0.7572 | 0.8511 |
| + Новые признаки, масштабирование и измененные гиперпараметры | 0.7635 | 0.8485 |
| **+ Увеличенная тестовая выборка** | **0.7702** | **0.8547** |

Градиентный бустинг

| Модель | F1-мера | PR AUC |
|----------|-------|--------|
| Бейзлайн | 0.7577 | 0.8423 |
| **+ Новые признаки, масштабирование, увеличенная тестовая выборка** | **0.7719** | **0.8560** |
| + Настройка гиперпараметров | 0.7719 | 0.8560 |
| Бейзлайн имплементация | 0.7183 | 0.8309 |
| **Улучшенный бейзлайн** | **0.7705** | **0.8558** |


В лабораторных работах исследовался датасет, содержащий данные о студентах. Решалась задача бинарной классификацией трудоустроен / не трудоустроен. Были исследованы пять алгоритмов обучения. Для улучшения показания метрик проводились различные улучшения: масштабирование признаков, создание новых признаков, измененение размера тестовой выборки, подбор гиперпараметров. В случае с каждым алгоритмом был свой оптимальный набор улучшений, хотя в случае KNN наилучшим вариантом был бейзлайн. Среди новых признаков наиболее полезным оказался Average_Marks . Все остальные новые признаки не давали эффекта самостоятельно, но в наборе новых признаков давали суммарно улучшение результатов. Также я отметила, что на качество предсказание может повлиять порядок признаков, для меня это было неожиданно. В нескольких алгоритмах ручной подбор гиперпараметров давал лучший результат по сравнению с автоподбором. Все модели показывают достаточно высокие метрики, что говорит о хорошем качестве предсказаний. Наилучшие результаты демонстрирует KNN, ансамблевые методы и логистическая регрессия на одном уровне. Решающее дерево предсказывает хуже всего, однако интересно отметить, что разница между улучшенной версией и безлайном выше всего именно у этого алгоритма.

## Регрессия

KNN

| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Библиотечный KNN (бейзлайн) | 0.370 | 2.159 | 2.876 |
| **Библиотечный KNN (улучшенный)** | **0.509** | 2.181 | 2.539 |
| Имплементированный KNN (бейзлайн) | 0.431 | 2.085 | 2.510 |
| **Имплементированный KNN (улучшенный)** | **0.533** | **1.914** | **2.273** |

Линейная регрессия

| Модель                              | R²      | MAE     | RMSE    |
|------------------------------------|---------|---------|---------|
| Бейзлайн (LinearRegression)        | 0.2978  | 1.9939  | 2.7879  |
| **Улучшенный бейзлайн (Ridge)**        | **0.6580**  | **1.7189**  | **2.1926**  |
| Имплементация бейзлайн (LinearRegression) | -6.4350 | 9.6313  | 10.2230 |
| Улучшенная имплементация (Ridge)   | 0.5927  | 1.7208  | 2.3412  |

Решающее дерево

| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Бейзлайн | -0.643 | 3.524 | 4.265 |
| **+ Фиксированные параметры** | **0.2765** | **2.2952** | **2.8299** |
| + Масштабирование | 0.2765 | 2.2952 | 2.8299 |
| + Подбор гиперпараметров | 0.1326 | 2.5582 | 3.0987 |
| Имплементация | -0.1763 | 3.0010 | 3.6085 |
| Имплементация с подходящими параметрами | 0.2431 | 2.2594 | 2.8945 |

Случайный лес

| Модель | R² | MAE | RMSE |
| --- | --- | --- | --- |
| Бейзлайн | 0.523 | 1.400 | 1.906 |
| + Удаление пропущенных значений | 0.701 | 1.632 | 1.980 |
| **+ Новые признаки** | **0.840** | **1.093** | **1.450** |
| + Фиксированные гиперпараметры | 0.747 | 1.350 | 1.820 |
| + Автоподбор гиперпараметров | 0.755 | 1.337 | 1.794 |
| Имплементация бейзлайн | 0.696 | 1.574 | 1.997 |
| + Новые признаки | 0.776 | 1.343 | 1.715 |
| С гиперпараметрами (без новых признаков) | 0.669 | 1.682 | 2.081 |
| С гиперпараметрами (с новыми признаками) | 0.798 | 1.217 | 1.627 |

Градиентный бустинг

| Модель | R² | MAE | RMSE |
|----------|-------|--------|--------|
| Бейзлайн | 0.3358 | 2.1913 | 2.7115 |
| + Новые признаки | 0.4002 | 2.0657 | 2.5767 |
| + Автоподбор гиперпараметров | 0.4604 | 1.8624 | 2.4439 |
| **+ Фиксированные гиперпараметры** | **0.5377** | **1.8571** | **2.2621** |
| Имплементация бейзлайн | 0.3341 | 2.2029 | 2.7149 |
| + Новые признаки | 0.3983 | 2.0763 | 2.5808 |
| **+ Автоподбор гиперпараметров** | **0.5463** | **1.8370** | **2.2410** |

Во второй части исследования алгоритмов использовался датасет с морфометрическими параметрами опоссумов. Решалась задача регрессии, предсказывался размер животного. Были исследованы пять алгоритмов обучения. Для
улучшения показаний метрик проводились различные улучшения: масштабирование
признаков, создание новых признаков, подбор гиперпараметров. В случае с каждым алгоритмом был свой оптимальный набор улучшений. Например, для случайного леса ключевым оказалось добавление новых признаков, а для градиентного бустинга и KNN — подбор гиперпараметров и масштабирование. Были изучены влияние различных новых признаков, показывающих отношение однив размеров к другим. Все модели, кроме решающего дерева, показывают приемлемые результаты, что говорит о наличии значимых закономерностей в данных, однако же качество предсказаний для регрессии ниже по сравнению с задачей классификации. Наилучшие результаты демонстрируют случайный лес и линейная регрессия с регуляризацией.